# Lanchain OpenAI Setup

In [1]:
import langchain
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Import chat templates
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Caching
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

In [2]:
cacheType = 'in_memory'

if cacheType == 'in_memory':
    set_llm_cache(InMemoryCache())
elif cacheType == 'sqlite':
    set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# Set OpenAI API key and create LLM and Chat LLM. Note that key can be stored in a separate file or as an environment variable. Refer to docs.
api_key = open('./openai_key.txt').read()
llm = OpenAI(openai_api_key=api_key)
chat = ChatOpenAI(openai_api_key=api_key)

In [20]:
# Step 1 - Import parser

# Import comma separated list parser from langchain
from langchain.output_parsers import (
    CommaSeparatedListOutputParser,
    DatetimeOutputParser
)

In [4]:
# Step 2 - format instructions
output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [5]:
sample_response = "foo, bar, baz"
output_parser.parse(sample_response)

['foo', 'bar', 'baz']

In [6]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [13]:
request = "give me 5 characteristics of dogs"
model_request = chat_prompt.format_prompt(request=request, format_instructions=output_parser.get_format_instructions()).to_messages()

In [16]:
result = chat(model_request)
print(f'Original output: {result.content}') # result.content
print(f'Parsed output: {output_parser.parse(result.content)}')

Original output: loyal, friendly, playful, protective, trainable
Parsed output: ['loyal', 'friendly', 'playful', 'protective', 'trainable']


### Handling incorrect parse responses

There are two main ways to solve this problem:
- System prompt - Have a strong system prompt to combine with your format instructions.
- OutputFixingParser - Using a chain, send your original reply to an LLM to try to fix it

In [21]:
datetime_parser = DatetimeOutputParser()
datetime_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0111-02-24T13:48:28.608735Z, 1824-09-06T13:25:32.595436Z, 2008-02-23T15:18:39.431396Z\n\nReturn ONLY this string, no other words!"

In [28]:
dt_template = "{request}\n{format_instructions}"
dt_prompt = HumanMessagePromptTemplate.from_template(dt_template)

# Using system prompt to guide the model
dt_system_template = "You only respond with a datetime string."
dt_system_prompt = SystemMessagePromptTemplate.from_template(dt_system_template)

dt_chat_prompt = ChatPromptTemplate.from_messages([dt_prompt, dt_system_prompt])

In [29]:
dt_request = "What data was the 13th Amendment oratified in the US?"
dt_model_request = dt_chat_prompt.format_prompt(request=dt_request, format_instructions=datetime_parser.get_format_instructions()).to_messages()

In [30]:
dt_result = chat(dt_model_request, temperature=0.0)
print(f'Original output: {dt_result.content}') # result.content
print(f'Parsed output: {datetime_parser.parse(dt_result.content)}')

Original output: 1865-12-06T00:00:00.000000Z
Parsed output: 1865-12-06 00:00:00


In [32]:
# Fix misformatted output using OutputFixingParser. This will reprompt the model to try and get it to fix the output.

from langchain.output_parsers import OutputFixingParser
misformatted = dt_result.content

new_parser = OutputFixingParser.from_llm(parser=datetime_parser, llm=chat)
fixed = new_parser.parse(misformatted)

print(fixed)

1865-12-06 00:00:00
